<h1>Sacramento's Best Trails</h1>
<h2>Data Visualization and Analysis (cont.)</h2>
<em>by:</em> Joy, Ruben, and Nancy

<h3>Let's find some trails for you!</h3>

> We'll help you find trails that you like.
> <br>Just enter some information and we will make you a map for your new adventure!

<h3>Step 1: Dependencies and Setup</h3>

In [53]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import ipywidgets as widgets

# Import API key
# Google API
from config import g_key

<h3>Step 2: Importing data from CSV</h3>

In [54]:
data = pd.read_csv("trails_dropped.csv")
#Removing first column (index)
data.drop(data.columns[0],axis=1,inplace=True)

In [55]:
#Dropping rows that don't have a valid value inside the difficulty column
indexNames = data[ (data['difficulty'] == 'missing') ].index
data.drop(indexNames , inplace=True)

data.loc[(data.difficulty == 'Easy'),'difficulty_level']=1
data.loc[(data.difficulty == 'Easy/Intermediate'),'difficulty_level']=2
data.loc[(data.difficulty == 'Intermediate'),'difficulty_level']=3
data.loc[(data.difficulty == 'Intermediate/Difficult'),'difficulty_level']=4
data.loc[(data.difficulty == 'Difficult'),'difficulty_level']=5
data.loc[(data.difficulty == 'Extremely Difficult'),'difficulty_level']=6

data['difficulty_level'] = data['difficulty_level'].astype('int32')

data = data.sort_values(by=['popular_vote'], ascending=False)

In [56]:
#Preview the data
data.head()

,id,name,type,summary,difficulty,stars,starVotes,location,length,ascent,descent,high,low,longitude,latitude,conditionStatus,conditionDetails,conditionDate,popular_vote,difficulty_level
234,7004777,Vernal and Nevada Falls Loop,Hike,A splendid loop past two of Yosemite's most po...,Intermediate/Difficult,4.8,251,"Yosemite Valley, California",6.2,1999,-1998,6022,4048,-119.5583,37.7326,All Clear,"Snowy, Icy - Some icy sections on JMT but rela...",2020-02-11 08:51:00,1204.8,4
235,7005207,Half Dome,Hike,THE premier route in Yosemite. Hike to the top...,Difficult,4.9,187,"Yosemite Valley, California",14.5,4457,-4457,8476,4083,-119.5583,37.7325,All Clear,NaN,2019-11-17 23:03:06,916.3,5
222,7004501,Lower Yosemite Fall Trail,Hike,Revel in stunning views of Yosemite Falls in i...,Easy,4.4,114,"Yosemite Valley, California",1.0,75,-84,4033,3964,-119.5962,37.7464,Unknown,NaN,1970-01-01 00:00:00,501.6,1
123,7001511,Tomales Point Trail,Hike,A trail with spectacular coastal views and pro...,Easy/Intermediate,4.6,69,"Inverness, California",9.8,996,-996,534,83,-122.9543,38.1891,All Clear,Mostly Dry,2020-01-27 15:29:26,317.4,2
228,7004770,Mirror Lake Loop,Hike,A beautiful loop through Tenaya Canyon.,Easy/Intermediate,3.9,70,"Yosemite Valley, California",4.7,290,-289,4160,3997,-119.5603,37.7393,Unknown,NaN,1970-01-01 00:00:00,273.0,2


<h3>Step 3: User input</h3>

> We will sort our data based on popularity and filter it to match your preference. Let's get started!

In [ ]:
v_name = input('Enter your name: ')

In [ ]:
v_number_of_trails = int(input('How many trails would you like to find? '))
v_difficulty = input('Enter level(s) difficulty: ')
v_difficulty = v_difficulty.split (",")

v_rating = float(input('Enter minimum rating(1.0-5.0): '))

Hi Joy!


In [59]:
yourTrails = data[(data.stars >= v_rating) & (data.difficulty.isin(v_difficulty))]
yourTrails = yourTrails.nlargest(v_number_of_trails,'popular_vote')
yourTrails

,id,name,type,summary,difficulty,stars,starVotes,location,length,ascent,descent,high,low,longitude,latitude,conditionStatus,conditionDetails,conditionDate,popular_vote,difficulty_level
222,7004501,Lower Yosemite Fall Trail,Hike,Revel in stunning views of Yosemite Falls in i...,Easy,4.4,114,"Yosemite Valley, California",1.0,75,-84,4033,3964,-119.5962,37.7464,Unknown,NaN,1970-01-01 00:00:00,501.6,1
123,7001511,Tomales Point Trail,Hike,A trail with spectacular coastal views and pro...,Easy/Intermediate,4.6,69,"Inverness, California",9.8,996,-996,534,83,-122.9543,38.1891,All Clear,Mostly Dry,2020-01-27 15:29:26,317.4,2
228,7004770,Mirror Lake Loop,Hike,A beautiful loop through Tenaya Canyon.,Easy/Intermediate,3.9,70,"Yosemite Valley, California",4.7,290,-289,4160,3997,-119.5603,37.7393,Unknown,NaN,1970-01-01 00:00:00,273.0,2
231,7009377,Sentinel Dome,Hike,An easy trail with some great westward sunset ...,Easy/Intermediate,4.7,55,"Yosemite Valley, California",2.1,350,-349,8065,7715,-119.5867,37.7129,Unknown,NaN,1970-01-01 00:00:00,258.5,2
218,7004499,Bridalveil Falls Trail,Hike,A short jaunt to one of Yosemite's most access...,Easy,4.3,60,"Yosemite Valley, California",0.4,98,-98,4052,3954,-119.6509,37.7168,All Clear,Dry,2020-02-01 08:34:49,258.0,1


<h3> Let's plot it on the map!</h3>

In [60]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [80]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'    
}
fig = gmaps.figure(layout=figure_layout, map_type = 'TERRAIN')

In [81]:
#Add the layers

coordinates = yourTrails[["latitude", "longitude"]].astype(float)

trail_info = yourTrails[["name", "location", "stars", "difficulty", "length", "ascent"]].values.tolist()

for i, s in enumerate(trail_info):
    trail_info[i] = f"""
    <b>Name</b>:<br>{s[0]}<br>
    <b>Location</b>:<br>{s[1]}<br>
    <b>Rating</b>:<br>{s[2]}<br>
    <b>Difficulty</b>:<br>{s[3]}<br>
    <b>Length</b>:<br>{s[4]} mi<br>
    <b>Elevation</b>:<br>{s[5]} ft<br>
    """

In [84]:
# Add marker layer
# Assign the marker layer to a variable

markers = gmaps.marker_layer(
        coordinates,
        info_box_content=trail_info
    )

# Add the layer to the map
fig.add_layer(markers)


In [83]:
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

><b>Now, save this map and get outside! :)</b>